# Create the Features and Labels Files
After calculating the lifted index, we will aggregate all the features and labels together and create one feature file and one label file.

In [8]:
import os
import glob
import pandas as pd

SILVER_GPH20S10K_PATH = '/Users/olievortex/lakehouse/default/Files/silver/igra2/gph20s10k'
SILVER_LI_PATH = '/Users/olievortex/lakehouse/default/Files/silver/igra2/li'
GOLD_PARQUET_PATH = '/Users/olievortex/lakehouse/default/Files/gold/igra2/liftedindex_lr'

# You will need the pyarrow package to create a parquet file
#%conda install pyarrow

In [9]:
# Make sure the destiation folder exists
os.makedirs(GOLD_PARQUET_PATH, exist_ok=True)


In [10]:
def df_from_concatinated_csv(path: str, pattern: str) -> pd.DataFrame:
    """Load multiple csv files into one DataFrame"""
    
    # Get the list of files from a folder
    file_list = glob.glob(path + '/' + pattern)

    # Read each CSV file into DataFrame
    # This creates a list of dataframes
    df_list = (pd.read_csv(file) for file in file_list)

    # Pandas doesn't like to concatinate "empty" DataFrames
    df_list = [dataframe for dataframe in df_list if dataframe.shape[0] > 0]

    # Concatenate all DataFrames
    return pd.concat(df_list, ignore_index=True)

def df_merge(features: pd.DataFrame, labels: pd.DataFrame):
    """Perform inner join on raw features and labels"""

    # Inner Join on id, effectie_date, and hour
    return features \
        .merge(labels, how='inner', on=['id', 'effective_date', 'hour'])

def orchestrate() -> pd.DataFrame:
    """Load and merge the raw files"""
    df_labels = df_from_concatinated_csv(SILVER_LI_PATH, '*-data-li.csv')
    df_features = df_from_concatinated_csv(SILVER_GPH20S10K_PATH, '*-data-gph20s10k.csv')
    return df_merge(df_features, df_labels)


In [11]:
# Get a combined DataFrame of the raw feauters and labels
df = orchestrate()

In [12]:
# Sanity checks
print(f"Row count: {df.shape[0]}")
print("df.info:")
df.info()

Row count: 336030
df.info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336030 entries, 0 to 336029
Columns: 131 entries, id to li
dtypes: float64(128), int64(1), object(2)
memory usage: 335.8+ MB


In [13]:
# View statistics
df.describe()

,hour,day_num,0_gph,0_pres,0_temp,0_dp,0_u,0_v,1_gph,1_pres,...,19_dp,19_u,19_v,20_gph,20_pres,20_temp,20_dp,20_u,20_v,li
count,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,...,336030.000000,336030.000000,336030.000000,336030.0,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000,336030.000000
mean,6.554748,-0.019505,511.762596,956.545331,13.043130,5.553043,-0.109772,0.430937,679.730111,937.312283,...,-55.341200,20.648738,-0.639877,10000.0,274.429911,-44.497937,-58.436341,21.710722,-0.494767,8.682874
std,6.101380,0.711816,397.491861,44.795354,12.567463,11.433532,2.771751,3.514093,277.989375,31.010859,...,10.201488,16.024596,16.309995,0.0,12.245019,7.105588,9.595212,16.433255,16.627336,10.513349
min,0.000000,-1.000000,14.000000,824.200000,-42.000000,-46.400000,-55.200000,-87.000000,513.300000,824.200000,...,-88.800000,-49.000000,-90.700000,10000.0,231.000000,-68.500000,-90.400000,-46.700000,-105.000000,-27.500000
25%,0.000000,-0.730000,230.000000,919.500000,3.800000,-3.100000,-1.900000,-1.700000,513.300000,919.500000,...,-62.100000,9.400000,-9.500000,10000.0,266.400000,-50.000000,-64.700000,10.200000,-9.700000,-0.100000
50%,12.000000,-0.030000,357.000000,972.800000,14.400000,6.000000,-0.000000,0.300000,513.300000,952.500000,...,-55.400000,19.600000,-0.500000,10000.0,276.200000,-44.600000,-58.700000,20.700000,-0.400000,7.400000
75%,12.000000,0.690000,849.000000,988.300000,22.800000,15.200000,1.500000,2.600000,849.000000,957.800000,...,-48.200000,31.000000,8.600000,10000.0,283.500000,-38.900000,-51.800000,32.400000,9.000000,16.300000
max,23.000000,1.000000,1475.000000,1042.700000,43.700000,37.100000,38.600000,46.300000,1475.000000,986.800000,...,-5.000000,108.700000,80.500000,10000.0,520.200000,-4.800000,-6.000000,107.800000,128.600000,57.300000


In [14]:
# Write the parquet file.
df.to_parquet(GOLD_PARQUET_PATH + '/gph20s10k_li.parquet')
